In [3]:

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

data = pd.read_csv('data/housing.csv')
data = pd.get_dummies(data)
x_train, x_test, y_train, y_test = train_test_split(data.drop(['SalePrice'], axis=1), 
                                                    data.SalePrice, test_size=0.3, 
                                                    random_state=0)
print(x_train)

        Id  MSSubClass  LotFrontage  LotArea  OverallQual  OverallCond  \
64      65          60    70.049958     9375            7            5   
682    683         120    70.049958     2887            6            5   
960    961          20    50.000000     7207            5            7   
1384  1385          50    60.000000     9060            6            5   
1100  1101          30    60.000000     8400            2            5   
...    ...         ...          ...      ...          ...          ...   
763    764          60    82.000000     9430            8            5   
835    836          20    60.000000     9600            4            7   
1216  1217          90    68.000000     8930            6            5   
559    560         120    70.049958     3196            7            5   
684    685          60    58.000000    16770            7            5   

      YearBuilt  YearRemodAdd  BsmtFinSF1  BsmtFinSF2  ...  SaleType_ConLw  \
64         1997          1998    

# 遗传算法

遗传算法是用于搜索非常大空间的全局优化技术。您可以将它们视为一种随机搜索。他们受到自然选择和繁殖的生物学机制的启发。

它们在可能的解决方案群体（称为世代）中工作，其中搜索空间中的每个解决方案都表示为一些有限符号集上的有限长度字符串（染色体），然后使用目标（或适应度）函数来评估适用性每个解决方案。

在特征选择方面，每条染色体将代表一个特征子集，并用二进制编码表示：1 表示“选择”给定特征，0 表示“不选择”特征。

> 例如，字符串 1001 表示选择第一个和最后一个特征作为特征子集。

之后，我们进行多次迭代，使用一些运算符从当前一代中创建新一代（新特征子集）可能的解决方案：

* Selection：概率性地过滤掉性能不佳的解决方案，同时选择高性能的解决方案进行利用。
* Cross Over：这是探索新解决方案和在特征之间交换信息的 GA 方式。该算子将用于随机选择染色体对，概率等于给定的交叉率。这样，我们生成了新的染色体，希望能保留前几代的良好特征。
* Mutation：这个操作保护 GAs 免受不可恢复的良好解决方案特征的损失。它以等于非常低的给定突变率的概率更改某些染色体的符号，以恢复丢失的遗传物质。

详细步骤：
1. 用随机生成的个体初始化种群——在我们的例子中，这意味着不同的特征子集——并创建一个机器学习算法。
2. 根据所选算法，使用您选择的评估指标评估每个特征子集的适应度。
3. 在新种群中重现高适应度染色体（特征子集）
4. 去除适应性差的染色体（Selection）
5. 构建新的染色体 (crossover).
6. 恢复丢失的特征 (mutation).
7. 重复步骤 2-6，直到满足停止条件（例如，可以是迭代次数）

GeneticSelectionCV:

* estimator: 用于评估特征子集适用性的模型，以及评估指标。
cv: int, 生成器，或用于确定交叉验证拆分策略的迭代器。
scoring: 评估指标，或者我们可以称之为评估染色体性能的适应度函数，即机器学习模型对特征子集的性能。
max_features: 定义最大特征数量。
n_population: 遗传算法的种群数量，表示不同的特征子集
crossover_proba: 遗传算法的交叉操作的概率值。
mutation_proba: 遗传算法变异操作的概率值。
n_generations: 一个整数，描述遗传算法的代数——停止标准。
crossover_independent_proba: 每个属性被交换的独立概率——这样我们为遗传算法的搜索提供了更大的灵活性。
mutation_independent_proba: 每个属性被遗传算法变异的独立概率。
n_gen_no_change: 如果最好的个体没有改变，则终止搜索所需的世代数。

In [3]:
from genetic_selection import GeneticSelectionCV
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=231)

selection = GeneticSelectionCV(model,
                              cv=5,
                              scoring="accuracy",
                              max_features=12,
                              n_population=120,
                              crossover_proba=0.5,
                              mutation_proba=0.2,
                              n_generations=50,
                              crossover_independent_proba=0.5,
                              mutation_independent_proba=0.05,
                              n_gen_no_change=10,
                              n_jobs=-1)

# fit the GA search to our data.
selection = selection.fit(x_train, y_train)

# print the results.
print(selection.support_)

[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False  True False False False
 False False False False False  True False False False  True False False
 False False False False False  True False False  True False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False  True False False False False False Fa

# 特征重要性
## 排列重要性

In [23]:
from sklearn.metrics import roc_auc_score

data = pd.read_csv('data/titanic.csv')
x_train, x_test, y_train, y_test = train_test_split(data.drop(['Survived'], axis=1), 
                                                    data.Survived, test_size=0.3, 
                                                    random_state=0)
numerical_x_train = x_train[x_train.select_dtypes([np.number]).columns].fillna(0)
# build your model and train it.
model = RandomForestClassifier(n_estimators=221)
model.fit(numerical_x_train, y_train)

# get the default score.
predict_y = model.predict(numerical_x_train)
train_auc = roc_auc_score(y_train, predict_y)
feature_dict_scores = {}

# loop over all the features
for feature in numerical_x_train.columns:

    # copy the dataset, because we want to do some permutation.
    x_train_copy = numerical_x_train.copy().reset_index(drop=True)
    y_train_copy = y_train.copy().reset_index(drop=True)
    
    # shuffle an individual feature.
    x_train_copy[feature] = x_train_copy[feature].sample(frac=1,random_state=2022).reset_index(drop=True)

    
    # make a prediction with permuted feature and calculate roc auc
    shuff_auc = roc_auc_score(y_train_copy, model.predict(x_train_copy))
    
    # save the drop in dictionary.
    feature_dict_scores[feature] = (train_auc - shuff_auc)
    
# print the resulting dictionary , feature => how much it drop the score.
print(feature_dict_scores)

{'PassengerId': 0.21194768009370735, 'Pclass': 0.21797791804949995, 'Age': 0.20932300817769678, 'SibSp': 0.09590898245157375, 'Parch': 0.0444784277998308, 'Fare': 0.2959317585301837}


## 深度学习-AEFS



In [28]:
# import the keras library.
from keras.layers import Dense
from keras.models import Sequential

numerical_x_train = numerical_x_train.astype('float64')
numerical_x_test = x_test[x_test.select_dtypes([np.number]).columns].fillna(0).astype('float64')
# the dimension of the encoding layer.
encoding_dim = 16

# create the autoencoder.
model = Sequential()

# add the encoding layer.
model.add(Dense(encoding_dim, activation='relu', input_shape=(numerical_x_train.shape[1],)))

# add the output layer.
model.add(Dense(numerical_x_train.shape[1], activation='linear'))

# compile the model, you can use whatever optimizer.
model.compile(optimizer='adadelta', loss='categorical_crossentropy')

# fit your model to the training data.
model.fit(numerical_x_train, numerical_x_train,
                epochs=50,
                batch_size=64,
                shuffle=True,
                validation_data=(numerical_x_test, numerical_x_test))

# get the first layer weights.
weights = model.layers[1].get_weights()[0]

# get the feature importance.
print(weights.sum(axis = 1))

Epoch 1/50
10/10 [==============================] - 1s 19ms/step - loss: 1217.7031 - val_loss: 1169.1609
Epoch 2/50
10/10 [==============================] - 0s 4ms/step - loss: 1216.3784 - val_loss: 1168.0691
Epoch 3/50
10/10 [==============================] - 0s 4ms/step - loss: 1215.0660 - val_loss: 1166.9574
Epoch 4/50
10/10 [==============================] - 0s 4ms/step - loss: 1213.7545 - val_loss: 1165.8386
Epoch 5/50
10/10 [==============================] - 0s 4ms/step - loss: 1212.4463 - val_loss: 1164.7054
Epoch 6/50
10/10 [==============================] - 0s 4ms/step - loss: 1211.1356 - val_loss: 1163.5625
Epoch 7/50
10/10 [==============================] - 0s 4ms/step - loss: 1209.8276 - val_loss: 1162.4102
Epoch 8/50
10/10 [==============================] - 0s 4ms/step - loss: 1208.5099 - val_loss: 1161.2482
Epoch 9/50
10/10 [==============================] - 0s 4ms/step - loss: 1207.1895 - val_loss: 1160.0618
Epoch 10/50
10/10 [==============================] - 0s 5ms/ste